In [5]:
!pip install tabulate
!pip install z3-solver

In [8]:
from ast import literal_eval
from tabulate import tabulate
import json
from z3 import *
import pandas as pd
import numpy as np

In [9]:
#Load the Crossword to solve

with open('mini_nyt_dec16.json', 'r') as f:
    crossword_to_solve = f.read()
crossword_to_solve = literal_eval(str(crossword_to_solve))
crossword_to_solve

{'Milk option': {'start': (0, 1),
  'direction': 'A',
  'length': 4,
  'answer': 'SKIM'},
 '5x5 crossword puzzle': {'start': (1, 1),
  'direction': 'A',
  'length': 4,
  'answer': 'MINI'},
 '___ Monsoon, drag queen who has won two seasons of "RuPauls Drag Race"': {'start': (2,
   0),
  'direction': 'A',
  'length': 5,
  'answer': 'JINKX'},
 'Word in some shoppe names': {'start': (3, 0),
  'direction': 'A',
  'length': 4,
  'answer': 'OLDE'},
 'Bit on a friendship bracele': {'start': (4, 0),
  'direction': 'A',
  'length': 4,
  'answer': 'BEAD'},
 '"The ) of :-)"': {'start': (0, 1),
  'direction': 'D',
  'length': 5,
  'answer': 'SMILE'},
 'Sorta': {'start': (0, 2), 'direction': 'D', 'length': 5, 'answer': 'KINDA'},
 'Signed, as a contract': {'start': (0, 3),
  'direction': 'D',
  'length': 5,
  'answer': 'INKED'},
 'A little bit of this, a little bit of that': {'start': (0, 4),
  'direction': 'D',
  'length': 3,
  'answer': 'MIX'},
 'What pays the bills': {'start': (2, 0),
  'direction

In [16]:
#Load the hints to solve

# with open("mini_nyt_dec16_hint_raw.json", 'r') as f:
#     crossword_hints = f.read()
# crossword_hints = literal_eval(str(crossword_hints))
# crossword_hints

with open("mini_nyt_dec16_hint.json", 'r') as f:
    crossword_hints = f.read()
crossword_hints = literal_eval(str(crossword_hints))
crossword_hints

{'Milk option': {'AABB': 109,
  'ESCS': 224,
  'ALYN': 431,
  'TMNT': 436,
  'USIA': 500,
  'IVAR': 510,
  'BCDE': 518,
  'CMAS': 526,
  'SEVE': 527,
  'CERN': 597,
  'SKIM': 0},
 '5x5 crossword puzzle': {'RAFA': 12,
  'LINC': 49,
  'TAMI': 63,
  'BAYH': 189,
  'ALYN': 196,
  'HERC': 239,
  'MOSE': 277,
  'FALK': 287,
  'RULA': 304,
  'ARYA': 307,
  'MINI': 0},
 '___ Monsoon, drag queen who has won two seasons of "RuPauls Drag Race"': {'LOHAN': 0,
  'ARKIN': 85,
  'CROCE': 109,
  'TRACI': 129,
  'JAFFE': 153,
  'HEDER': 188,
  'ALANA': 192,
  'IRVIN': 223,
  'JONZE': 241,
  'LIDDY': 313,
  'JINKX': 0},
 'Word in some shoppe names': {'ATOI': 29,
  'AHEN': 78,
  'EUSE': 94,
  'ASON': 198,
  'ESCS': 263,
  "IT'S": 304,
  'IBOS': 364,
  'OTOH': 372,
  'ETHS': 439,
  'EDHS': 453,
  'OLDE': 0},
 'Bit on a friendship bracele': {'GERI': 9,
  'KONY': 36,
  'ESCS': 47,
  'LUCI': 69,
  'ATCO': 79,
  'CERE': 168,
  'LACI': 349,
  'SHEL': 359,
  'RALF': 418,
  'DIAN': 419,
  'BEAD': 0},
 '"The ) of

In [17]:
CROSSWORD_GRID = crossword_to_solve
CROSSWORD_GRID_HINTS = crossword_hints

In [21]:
class Crossword():
    def __init__(self):
        pass

    def apply_constraints(self):
        z3_solver = z3.Solver()
        z3_solver.add(self.apply_faiss_answer_constraint())
        z3_solver.add(self.apply_RC_constraint())
        chk = z3_solver.check()
        if str(chk) == 'unsat': #Unstatisfied
            print("Unsolved: Provided answers do not satisfy the Crossword Puzzle.")
            return None
        else:
            print("Solved: Provided answers satisfy the Crossword Puzzle.")
            return z3_solver.model() #Satisfied
        return None
    
    def apply_faiss_answer_constraint(self):
        clues = self.convert_clues_code()
        matrix, start_positions, max_val = self.get_grid()

        clue_constraint = list()

        for x_index in range(max_val):
            for y_index in range(max_val):
                if (x_index, y_index) in list(start_positions.keys()):
                    pos_info = CROSSWORD_GRID[start_positions[(x_index, y_index)]]
                    clue = start_positions[(x_index, y_index)]

                    guesses_ord = clues[clue]
                    all_guesses_constraint = list()

                    for guess_ord in guesses_ord:
                        guess_constraint = list()
                        x_i = x_index
                        y_i = y_index

                        for ch in guess_ord:
                            if isinstance(matrix[x_i][y_i], tuple):
                                # NOTE: _1 -> signifies the COLUMN index AND _2 -> signifies the ROW index
                                if pos_info["direction"] == "D":
                                    matrix_val = matrix[x_i][y_i][0]
                                elif pos_info["direction"] == "A":
                                    matrix_val = matrix[x_i][y_i][1]
                            else:
                                matrix_val = matrix[x_i][y_i]

                            guess_constraint.append(z3.And(matrix_val == ch))

                            if pos_info["direction"] == "D":
                                x_i += 1
                            elif pos_info["direction"] == "A":
                                y_i += 1

                        all_guesses_constraint.append(z3.And(guess_constraint))

                    clue_constraint.append(z3.Or(all_guesses_constraint))

        clues_constraint = z3.And(clue_constraint)

        return clues_constraint

    def apply_RC_constraint(self):
        clues = self.convert_clues_code()
        matrix, start_positions, max_val = self.get_grid()

        equality_constraint = list()

        for x_index in range(max_val):
            for y_index in range(max_val):
                if isinstance(matrix[x_index][y_index], tuple):
                    first = matrix[x_index][y_index][0]
                    second = matrix[x_index][y_index][1]

                    equality_constraint.append(z3.And(first == second))

        apply_RC_constraint = z3.And(equality_constraint)

        return apply_RC_constraint  

    def convert_clues_code(self):
        """converts all the clues (in string format) to a list of their ascii characters value
           example: "happy" -> [109, 117, 109, 98, 97, 105]
        """
        clues = self.get_hints()
        clues_ord = dict()

        for clue in clues:
            for guess in clues[clue]:
                try:
                    clues_ord[clue].append([ord(ch) for ch in guess.lower()])
                except:
                    clues_ord[clue] = [[ord(ch) for ch in guess.lower()]]

        return clues_ord    

    # Clues obtained from Graph DB
    def get_hints(self):
        hints = CROSSWORD_GRID_HINTS
        return hints

    def get_grid(self):
        clues = self.get_hints()
        start_positions = dict()

        # finding the square matrix size
        max_val = 0
        for clue in CROSSWORD_GRID:
            start_positions[CROSSWORD_GRID[clue]["start"]] = clue

            if CROSSWORD_GRID[clue]["start"][0] > max_val:
                max_val = CROSSWORD_GRID[clue]["start"][0]
            elif CROSSWORD_GRID[clue]["start"][1] > max_val:
                max_val = CROSSWORD_GRID[clue]["start"][1]

        # As the CROSSWORD_GRID starts from 0, we have to add 1 to get actual max_val
        max_val += 1

        matrix = [[None for index in range(max_val)] for index in range(max_val)]
        
        tabulate(matrix)

        # The following code encodes the matrix declared with z3.Int(VALUE)
        # If at a position value is: (z3.Int(VALUE)_1, z3.Int(VALUE)_2), it signifies intersection is taking place there
        # NOTE: _1 -> signifies the COLUMN index AND _2 -> signifies the ROW index
        for x_index in range(max_val):
            for y_index in range(max_val):

                if (x_index, y_index) in list(start_positions.keys()):
                    pos_info = CROSSWORD_GRID[start_positions[(x_index, y_index)]]
                    for i in range(pos_info["length"]):

                        if pos_info["direction"] == "A":
                            if isinstance(matrix[x_index][y_index + i], z3.z3.ArithRef):
                                matrix[x_index][y_index + i] = (z3.Int("alpha_" + str(x_index) + "_" + str(y_index + i) + "_1"), z3.Int("alpha_" + str(x_index) + "_" + str(y_index + i) + "_2"))
                            else:
                                matrix[x_index][y_index + i] = z3.Int("alpha_" + str(x_index) + "_" + str(y_index + i))

                        elif pos_info["direction"] == "D":
                            if isinstance(matrix[x_index + i][y_index], z3.z3.ArithRef):
                                matrix[x_index + i][y_index] = (z3.Int("alpha_" + str(x_index + i) + "_" + str(y_index) + "_1"), z3.Int("alpha_" + str(x_index + i) + "_" + str(y_index) + "_2"))
                            else:
                                matrix[x_index + i][y_index] = z3.Int("alpha_" + str(x_index + i) + "_" + str(y_index))

        return matrix, start_positions, max_val

    def solution(self):
        solved = self.apply_constraints()
        if solved is not None:
            solved_keys = [index.name() for index in solved]
            matrix, start_positions, max_val = self.get_grid()

            for x_index in range(max_val):
                for y_index in range(max_val):
                    if isinstance(matrix[x_index][y_index], tuple):
                        matrix_val = matrix[x_index][y_index][0]
                    elif matrix[x_index][y_index] != None:
                        matrix_val = matrix[x_index][y_index]
                    else:
                        continue

                    no = solved[matrix_val]
                    ch = chr(no.as_long())
                    matrix[x_index][y_index] = ch
        else:
            matrix = None

        return matrix    

In [22]:
c = Crossword()
c.solution()

Solved: Provided answers satisfy the Crossword Puzzle.


[[None, 's', 'k', 'i', 'm'],
 [None, 'm', 'i', 'n', 'i'],
 ['j', 'i', 'n', 'k', 'x'],
 ['o', 'l', 'd', 'e', None],
 ['b', 'e', 'a', 'd', None]]